In [2]:
import pandas
import numpy
from numpy import std, correlate, sqrt
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.stats.mediation import Mediation
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

### Preparing data

In [3]:
filepath = 'D:/DEA/Data/Original_data.txt'
data = pandas.read_csv(filepath,sep='\t')
data.rename(columns={'origi':'Id'},inplace=True)
print(data.shape,'from 2019_05_25_Results.xlxs')
print(data.Year.unique(),'Years')
print(data.Id.nunique(),'Number of firms')
filepath = 'D:/DEA/Data/Database_secondreviewJIBS.dta' #
data1 = pandas.read_stata(filepath)
data1.rename(columns={'id':'Id','year':'Year','country':'Country'},inplace=True)
data1 = data1[['Id','Year','Country']]
data1['Year'] = pandas.to_numeric(data1.Year.astype(str).str[0:4])
print(data1.shape,'to get country')
data = pandas.merge(left=data,right=data1,how='left',on=['Year','Id'])
print(data.Country.unique())
filepath = 'D:/DEA/Data/2019_05_25_Results_updated.txt'
data2 = pandas.read_csv(filepath,sep='\t')
print(data2.shape,'correction by Elio of market commonality')
data2.rename(columns={'origi':'Id'},inplace=True)
data2 = data2[['Id','Year','Market_commonality']]
data.drop('Market_commonality',axis=1,inplace = True)
data = pandas.merge(left=data,right=data2,how='inner',on=['Year','Id'])
print(data.Non_RD_alliances.sum(),'Total number of Non-RD alliances')
print(data.RD_alliances.sum(),'Total number of RD alliances')

(1232, 33) from 2019_05_25_Results.xlxs
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012] Years
56 Number of firms
(1539, 3) to get country
[nan 'USA' 'Switzerland' 'Netherlands' 'Germany' 'South Africa' 'Japan'
 'United Kingdom' 'France' 'Australia' 'Ireland' 'Spain' 'Denmark'
 'Sweden']
(1232, 34) correction by Elio of market commonality
4089.0 Total number of Non-RD alliances
7676.0 Total number of RD alliances


### Creating variables

In [4]:
df = data.copy()
df.reset_index(inplace=True,drop=True)
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)
# ASPIRATIONS
df['SA']=df.groupby(['Year']).ROE.transform('sum')-df.ROE
df['n']=df.groupby(['Year']).ROE.transform('count')-1
df['SA']=df['SA']/df['n']
df['distance']=df['ROE']-df['SA']
df.loc[df.distance<0,'BSA']=df['distance']
df.loc[(df.distance>0)|(df.distance==0),'BSA']=0
df.loc[(df.distance>0)|(df.distance==0),'ASA']=df['distance']
df.loc[df.distance<0,'ASA']=0
df['BSA'] = df.groupby('Id')['BSA'].shift()
# DURATION
df['DURATION'] = 0
mylist = []
for i in range(df.Id.nunique()):
  mylist.append(df.loc[df.Id==i+1,['Id','Year','BSA','DURATION']].values.tolist())
m = [0]
for firm in range(df.Id.nunique()):
  for i in mylist[firm]: 
    if i[2]<0:
      m.append([i[0],i[1],m[-1][2]+1])
    else:
      m.append([i[0],i[1],i[3]])
del m[0]
print(len(m))
mydf = pandas.DataFrame(m,columns=['Id','Year','DURATION'])
df = df.drop('DURATION',axis=1)
df = pandas.merge(left=df,right=mydf,how='inner',on=['Year','Id'])
print(df.loc[(df.Id==38)|(df.Id==39),['Id','Year','BSA','DURATION']])
# ATTENTION
filepath = 'D:/DEA/Data/Results_def_01_for_Elio.txt'
data5 = pandas.read_csv(filepath,sep='\t')
data5 = data5.iloc[1:1008,[0,1,25,26,27]]
data5.columns = ['Year','Id','PRODUCT','GEOGRAPHY','GROWTH']
data5['Year'] = pandas.to_numeric(data5.Year.astype(str).str[0:4])
df = pandas.merge(left=df,right=data5,how='inner',on=['Year','Id'])
# R&D SEARCH
print(df.Non_RD_alliances.corr(df.RD_alliances),'correlation between RD alliances and non-RD alliances')
df['RD'] = df.RD_alliances
# SIZE
df['Size'] = numpy.log(df.Employees) 
df

1232
     Id  Year       BSA  DURATION
814  38  1991       NaN       0.0
815  38  1992  0.000000       0.0
816  38  1993  0.000000       0.0
817  38  1994  0.000000       0.0
818  38  1995  0.000000       0.0
819  38  1996 -0.014711       1.0
820  38  1997 -0.099922       2.0
821  38  1998  0.000000       0.0
822  38  1999 -0.213096       1.0
823  38  2000 -0.060731       2.0
824  38  2001 -0.127189       3.0
825  38  2002 -0.009038       4.0
826  38  2003  0.000000       0.0
827  38  2004  0.000000       0.0
828  38  2005 -0.046308       1.0
829  38  2006 -0.000654       2.0
830  38  2007 -0.000157       3.0
831  38  2008 -0.646780       4.0
832  38  2009 -0.264653       5.0
833  38  2010 -0.167467       6.0
834  38  2011 -0.091477       7.0
835  38  2012 -0.106878       8.0
836  39  1991       NaN       0.0
837  39  1992       NaN       0.0
838  39  1993       NaN       0.0
839  39  1994       NaN       0.0
840  39  1995       NaN       0.0
841  39  1996       NaN       0.0
842  39  

,Id,Year,Age,Employees,Total_assets,Net_sales,Human_resources_1,Human_resources_2,Physical_resources_1,Physical_resources_2,...,n,distance,BSA,ASA,DURATION,PRODUCT,GEOGRAPHY,GROWTH,RD,Size
0,1,1996,94.0,74289.0,13364.000,14236.000,0.192,5.218,4844.000,0.340,...,51,0.049961,-0.039244,0.049961,1.0,0.0,0.784127,0.215873,0.0,11.215718
1,1,1997,95.0,75639.0,13238.000,15070.000,0.199,5.019,5034.000,0.334,...,51,0.232902,0.000000,0.232902,0.0,0.0,0.774649,0.225351,0.0,11.233727
2,1,1998,96.0,73564.0,14153.000,15021.000,0.204,4.897,5566.000,0.371,...,52,-0.122385,0.000000,0.000000,0.0,0.0,0.750883,0.249117,2.0,11.205911
3,1,1999,97.0,70549.0,13896.000,15659.000,0.222,4.505,5656.000,0.361,...,52,0.094192,-0.122385,0.094192,1.0,0.0,0.793790,0.206210,0.0,11.164063
4,1,2000,98.0,75000.0,14522.000,16724.000,0.223,4.485,5823.000,0.348,...,53,0.117340,0.000000,0.117340,0.0,0.0,0.788899,0.211101,0.0,11.225243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,56,2008,148.0,47426.0,44031.724,22833.908,0.481,2.077,11198.211,0.490,...,49,0.038408,-0.048040,0.038408,1.0,NaN,NaN,NaN,4.0,10.766926
1003,56,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45,NaN,0.000000,NaN,0.0,NaN,NaN,NaN,4.0,NaN
1004,56,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,44,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1005,56,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


### Correlation

In [5]:
mydata = df[['RD','BSA','DURATION','PRODUCT','GROWTH','GEOGRAPHY','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
print(len(mydata),'Number of observations')
ds_mean = mydata.agg(['mean','std']).T
corr = mydata.corr()
corr = pandas.concat([ds_mean,corr], axis = 1)
corr = corr.applymap('{:.2f}'.format)
corr['statistics'] = ' '
corr.set_index('statistics',append=True,inplace=True)
print(corr)

def pearsonr_pval(x,y):
  return pearsonr(x,y)[1].round(3)
pval = mydata.corr(method=pearsonr_pval)
pval = pval.applymap('{:.3f}'.format)
pval = pval.applymap(lambda x: '(' + x + ')')
pval['statistics'] = 'p-value'
pval.set_index('statistics',append=True,inplace=True)

corr = pandas.concat([corr,pval],axis=0).sort_index(kind='merge').reindex(['R&D','BSA','DURATION','PRODUCT','GROWTH','GEOGRAPHY','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year'],axis=0,level=0)
filepath = 'D:/DEA/DEA-2/Correlation.txt'
corr.to_csv(filepath,sep = '\t',header=False,index=False)
corr

1007 Number of observations
                                   mean    std     RD    BSA DURATION PRODUCT  \
                    statistics                                                  
RD                                 8.04   8.55   1.00   0.15    -0.22    0.02   
BSA                               -0.10   0.22   0.15   1.00    -0.23   -0.07   
DURATION                           2.96   4.61  -0.22  -0.23     1.00    0.15   
PRODUCT                            0.64   0.41   0.02  -0.07     0.15    1.00   
GROWTH                             0.15   0.16   0.06   0.08    -0.03   -0.62   
GEOGRAPHY                          0.21   0.33  -0.06   0.05    -0.17   -0.92   
ASA                                0.10   0.34   0.06  -0.15    -0.14   -0.10   
Absorbed_slack                     1.97  37.88  -0.04   0.01    -0.02    0.23   
Unabsorbed_slack                   1.39   1.99  -0.18  -0.22     0.08    0.18   
Absorptive_capacity                0.23   0.72  -0.07  -0.42     0.06    0.18   


mean    std       RD      BSA DURATION  \
                    statistics                                              
BSA                               -0.10   0.22     0.15     1.00    -0.23   
                    p-value         NaN    NaN  (0.000)  (1.000)  (0.000)   
DURATION                           2.96   4.61    -0.22    -0.23     1.00   
                    p-value         NaN    NaN  (0.000)  (0.000)  (1.000)   
PRODUCT                            0.64   0.41     0.02    -0.07     0.15   
                    p-value         NaN    NaN  (0.649)  (0.087)  (0.001)   
GROWTH                             0.15   0.16     0.06     0.08    -0.03   
                    p-value         NaN    NaN  (0.175)  (0.076)  (0.434)   
GEOGRAPHY                          0.21   0.33    -0.06     0.05    -0.17   
                    p-value         NaN    NaN  (0.220)  (0.215)  (0.000)   
ASA                                0.10   0.34     0.06    -0.15    -0.14   
                    p-value         NaN    NaN  (0.102)  (0.000)  (0.000)   
Absorbed_slack                     1.97  37.88    -0.04     0.01    -0.02   
                    p-value         NaN    NaN  (0.322)  (0.797)  (0.571)   
Unabsorbed_slack                   1.39   1.99    -0.18    -0.22     0.08   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.027)   
Absorptive_capacity                0.23   0.72    -0.07    -0.42     0.06   
                    p-value         NaN    NaN  (0.046)  (0.000)  (0.053)   
Market_commonality                 0.41   0.14     0.01    -0.09     0.05   
                    p-value         NaN    NaN  (0.771)  (0.009)  (0.150)   
Size                               9.37   1.76     0.41     0.34    -0.17   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
Year                            2003.51   5.19    -0.14     0.03     0.17   
                    p-value         NaN    NaN  (0.000)  (0.342)  (0.000)   

                                PRODUCT   GROWTH GEOGRAPHY      ASA  \
                    statistics                                        
BSA                               -0.07     0.08      0.05    -0.15   
                    p-value     (0.087)  (0.076)   (0.215)  (0.000)   
DURATION                           0.15    -0.03     -0.17    -0.14   
                    p-value     (0.001)  (0.434)   (0.000)  (0.000)   
PRODUCT                            1.00    -0.62     -0.92    -0.10   
                    p-value     (1.000)  (0.000)   (0.000)  (0.020)   
GROWTH                            -0.62     1.00      0.28     0.11   
                    p-value     (0.000)  (1.000)   (0.000)  (0.010)   
GEOGRAPHY                         -0.92     0.28      1.00     0.07   
                    p-value     (0.000)  (0.000)   (1.000)  (0.110)   
ASA                               -0.10     0.11      0.07     1.00   
                    p-value     (0.020)  (0.010)   (0.110)  (1.000)   
Absorbed_slack                     0.23    -0.19     -0.18    -0.01   
                    p-value     (0.000)  (0.000)   (0.000)  (0.722)   
Unabsorbed_slack                   0.18    -0.09     -0.17     0.10   
                    p-value     (0.000)  (0.070)   (0.000)  (0.007)   
Absorptive_capacity                0.18    -0.12     -0.17     0.13   
                    p-value     (0.000)  (0.004)   (0.000)  (0.000)   
Market_commonality                -0.12     0.12      0.08    -0.02   
                    p-value     (0.007)  (0.005)   (0.051)  (0.558)   
Size                              -0.28     0.18      0.26     0.04   
                    p-value     (0.000)  (0.000)   (0.000)  (0.294)   
Year                               0.01    -0.06      0.02    -0.03   
                    p-value     (0.883)  (0.175)   (0.632)  (0.421)   

                               Absorbed_slack Unabsorbed_slack  \
                    statistics                                   
BSA                                      0.01        

## Attention to growth

In [6]:
mydata = df[['Id','RD','PRODUCT','GROWTH','GEOGRAPHY','BSA','DURATION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
mediator_model = sm.OLS.from_formula('GROWTH~BSA+DURATION+C(Id)', data=mydata)
outcome_model = sm.OLS.from_formula('RD~BSA+DURATION+GROWTH+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)',data=mydata)        
mediator = mediator_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
outcome = outcome_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
print(mediator.summary())
print(outcome.summary())
BSA_Mediation = Mediation(outcome_model,mediator_model,'BSA','GROWTH').fit(n_rep=250)
print(BSA_Mediation.summary())
Duration_Mediation = Mediation(outcome_model,mediator_model,'DURATION','GROWTH').fit(n_rep=250)
print(Duration_Mediation.summary())

                            OLS Regression Results                            
Dep. Variable:                 GROWTH   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     185.5
Date:                Fri, 05 Mar 2021   Prob (F-statistic):           2.62e-36
Time:                        13:31:22   Log-Likelihood:                 614.30
No. Observations:                 374   AIC:                            -1141.
Df Residuals:                     330   BIC:                            -967.9
Df Model:                          43                                         
Covariance Type:            hac-panel                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.1971      0.009     22.900      

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.545474        0.004751        1.251653    0.048
ACME (treated)            0.545474        0.004751        1.251653    0.048
ADE (control)             3.897366       -0.025163        8.261306    0.056
ADE (treated)             3.897366       -0.025163        8.261306    0.056
Total effect              4.442839        0.368654        8.585126    0.032
Prop. mediated (control)  0.116328       -0.002295        0.568279    0.064
Prop. mediated (treated)  0.116328       -0.002295        0.568279    0.064
ACME (average)            0.545474        0.004751        1.251653    0.048
ADE (average)             3.897366       -0.025163        8.261306    0.056
Prop. mediated (average)  0.116328       -0.002295        0.568279    0.064
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.026863       -0.137831        0.067394    0.560
ACME (treate

## Attention to growth - Quadratic Aspirations

In [7]:
mydata = df[['Id','RD','PRODUCT','GROWTH','GEOGRAPHY','BSA','DURATION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = (mydata - mydata.mean()) / mydata.std() 
mydata['BSA2'] = mydata['BSA']*mydata['BSA']
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
mediator_model = sm.OLS.from_formula('GROWTH~BSA+BSA2+DURATION+C(Id)', data=mydata)
outcome_model = sm.OLS.from_formula('RD~BSA+BSA2+DURATION+GROWTH+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)',data=mydata)        
mediator = mediator_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
outcome = outcome_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
print(mediator.summary())
print(outcome.summary())
BSA_Mediation = Mediation(outcome_model,mediator_model,'BSA','GROWTH').fit(n_rep=250)
print(BSA_Mediation.summary())
Duration_Mediation = Mediation(outcome_model,mediator_model,'DURATION','GROWTH').fit(n_rep=250)
print(Duration_Mediation.summary())

                            OLS Regression Results                            
Dep. Variable:                 GROWTH   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     164.4
Date:                Fri, 05 Mar 2021   Prob (F-statistic):           2.67e-35
Time:                        13:32:15   Log-Likelihood:                -57.094
No. Observations:                 374   AIC:                             204.2
Df Residuals:                     329   BIC:                             380.8
Df Model:                          44                                         
Covariance Type:            hac-panel                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.015179       -0.000636        0.041066    0.056
ACME (treated)            0.015179       -0.000636        0.041066    0.056
ADE (control)             0.099331       -0.010404        0.215866    0.064
ADE (treated)             0.099331       -0.010404        0.215866    0.064
Total effect              0.114510        0.004850        0.237785    0.048
Prop. mediated (control)  0.109743       -0.078067        0.734329    0.088
Prop. mediated (treated)  0.109743       -0.078067        0.734329    0.088
ACME (average)            0.015179       -0.000636        0.041066    0.056
ADE (average)             0.099331       -0.010404        0.215866    0.064
Prop. mediated (average)  0.109743       -0.078067        0.734329    0.088
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.013054       -0.040822        0.004414    0.216
ACME (treate

## Attention to growth - Quadratic Duration

## Attention to -growth - 2 Quadratic terms

In [7]:
mydata = df[['Id','RD','PRODUCT','GROWTH','GEOGRAPHY','BSA','DURATION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = (mydata - mydata.mean()) / mydata.std() 
mydata['BSA2'] = mydata['BSA']*mydata['BSA']
mydata['DURATION2'] = mydata['DURATION']*mydata['DURATION']
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
mediator_model = sm.OLS.from_formula('GROWTH~BSA+BSA2+DURATION+DURATION2+C(Id)', data=mydata)
outcome_model = sm.OLS.from_formula('RD~BSA+BSA2+DURATION+DURATION2+GROWTH+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)',data=mydata)        
mediator = mediator_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
outcome = outcome_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
print(mediator.summary())
print(outcome.summary())
BSA_Mediation = Mediation(outcome_model,mediator_model,'BSA','GROWTH').fit(n_rep=250)
print(BSA_Mediation.summary())
Duration_Mediation = Mediation(outcome_model,mediator_model,'DURATION','GROWTH').fit(n_rep=250)
print(Duration_Mediation.summary())

                            OLS Regression Results                            
Dep. Variable:                 GROWTH   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     169.3
Date:                Fri, 05 Mar 2021   Prob (F-statistic):           1.31e-35
Time:                        13:16:56   Log-Likelihood:                -56.406
No. Observations:                 374   AIC:                             204.8
Df Residuals:                     328   BIC:                             385.3
Df Model:                          45                                         
Covariance Type:            hac-panel                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.017124       -0.001593        0.045939    0.072
ACME (treated)            0.017124       -0.001593        0.045939    0.072
ADE (control)             0.097161       -0.030229        0.235179    0.152
ADE (treated)             0.097161       -0.030229        0.235179    0.152
Total effect              0.114284       -0.011785        0.251070    0.088
Prop. mediated (control)  0.128259       -1.003637        1.440096    0.160
Prop. mediated (treated)  0.128259       -1.003637        1.440096    0.160
ACME (average)            0.017124       -0.001593        0.045939    0.072
ADE (average)             0.097161       -0.030229        0.235179    0.152
Prop. mediated (average)  0.128259       -1.003637        1.440096    0.160
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.005352       -0.035990        0.021602    0.624
ACME (treate

## Attention to product

In [61]:
mydata = df[['Id','RD','PRODUCT','GROWTH','GEOGRAPHY','BSA','DURATION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
mediator_model = sm.OLS.from_formula('PRODUCT~BSA+DURATION+C(Id)+C(Year)', data=mydata)
outcome_model = sm.OLS.from_formula('RD~BSA+DURATION+PRODUCT+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)+C(Year)',data=mydata)        
mediator = mediator_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
outcome = outcome_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
print(mediator.summary())
print(outcome.summary())
BSA_Mediation = Mediation(outcome_model,mediator_model,'BSA','PRODUCT').fit(n_rep=250)
print(BSA_Mediation.summary())
Duration_Mediation = Mediation(outcome_model,mediator_model,'DURATION','PRODUCT').fit(n_rep=250)
print(Duration_Mediation.summary())

                            OLS Regression Results                            
Dep. Variable:                PRODUCT   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 1.635e+04
Date:                Mon, 01 Mar 2021   Prob (F-statistic):           7.44e-77
Time:                        18:32:07   Log-Likelihood:                 731.55
No. Observations:                 374   AIC:                            -1341.
Df Residuals:                     313   BIC:                            -1102.
Df Model:                          60                                         
Covariance Type:            hac-panel                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0046      0.004     

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.010500       -0.534254        0.530300    0.992
ACME (treated)            0.010500       -0.534254        0.530300    0.992
ADE (control)             5.222587        1.496357        9.739662    0.024
ADE (treated)             5.222587        1.496357        9.739662    0.024
Total effect              5.233087        1.480612        9.711033    0.016
Prop. mediated (control)  0.000067       -0.125564        0.124834    0.992
Prop. mediated (treated)  0.000067       -0.125564        0.124834    0.992
ACME (average)            0.010500       -0.534254        0.530300    0.992
ADE (average)             5.222587        1.496357        9.739662    0.024
Prop. mediated (average)  0.000067       -0.125564        0.124834    0.992
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.000163       -0.060130        0.062855    0.992
ACME (treate

## Attention to geography

In [62]:
mydata = df[['Id','RD','PRODUCT','GROWTH','GEOGRAPHY','BSA','DURATION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
mediator_model = sm.OLS.from_formula('GEOGRAPHY~BSA+DURATION+C(Id)+C(Year)', data=mydata)
outcome_model = sm.OLS.from_formula('RD~BSA+DURATION+GEOGRAPHY+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)+C(Year)',data=mydata)        
mediator = mediator_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
outcome = outcome_model.fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})
print(mediator.summary())
print(outcome.summary())
BSA_Mediation = Mediation(outcome_model,mediator_model,'BSA','GEOGRAPHY').fit(n_rep=250)
print(BSA_Mediation.summary())
Duration_Mediation = Mediation(outcome_model,mediator_model,'DURATION','GEOGRAPHY').fit(n_rep=250)
print(Duration_Mediation.summary())

                            OLS Regression Results                            
Dep. Variable:              GEOGRAPHY   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     9760.
Date:                Mon, 01 Mar 2021   Prob (F-statistic):           2.92e-72
Time:                        18:34:14   Log-Likelihood:                 779.88
No. Observations:                 374   AIC:                            -1438.
Df Residuals:                     313   BIC:                            -1198.
Df Model:                          60                                         
Covariance Type:            hac-panel                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.7862      0.019     

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.091867       -0.188116        0.561961    0.640
ACME (treated)            0.091867       -0.188116        0.561961    0.640
ADE (control)             5.314003        1.452944        8.963105    0.024
ADE (treated)             5.314003        1.452944        8.963105    0.024
Total effect              5.405870        1.537322        9.191582    0.024
Prop. mediated (control)  0.008085       -0.048001        0.108134    0.664
Prop. mediated (treated)  0.008085       -0.048001        0.108134    0.664
ACME (average)            0.091867       -0.188116        0.561961    0.640
ADE (average)             5.314003        1.452944        8.963105    0.024
Prop. mediated (average)  0.008085       -0.048001        0.108134    0.664
                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -0.002691       -0.062884        0.041695    0.912
ACME (treate